In [56]:
"""
  @misc{https://doi.org/10.48550/arxiv.2201.12086,
    doi = {10.48550/ARXIV.2201.12086},
    
    url = {https://arxiv.org/abs/2201.12086},
    
    author = {Li, Junnan and Li, Dongxu and Xiong, Caiming and Hoi, Steven},
    
    keywords = {Computer Vision and Pattern Recognition (cs.CV), FOS: Computer and information sciences, FOS: Computer and information sciences},
    
    title = {BLIP: Bootstrapping Language-Image Pre-training for Unified Vision-Language Understanding and Generation},
    
    publisher = {arXiv},
    
    year = {2022},
    
    copyright = {Creative Commons Attribution 4.0 International}
  }
"""


'\n  @misc{https://doi.org/10.48550/arxiv.2201.12086,\n    doi = {10.48550/ARXIV.2201.12086},\n    \n    url = {https://arxiv.org/abs/2201.12086},\n    \n    author = {Li, Junnan and Li, Dongxu and Xiong, Caiming and Hoi, Steven},\n    \n    keywords = {Computer Vision and Pattern Recognition (cs.CV), FOS: Computer and information sciences, FOS: Computer and information sciences},\n    \n    title = {BLIP: Bootstrapping Language-Image Pre-training for Unified Vision-Language Understanding and Generation},\n    \n    publisher = {arXiv},\n    \n    year = {2022},\n    \n    copyright = {Creative Commons Attribution 4.0 International}\n  }\n'

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00


In [3]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a woman and her dog on the beach
woman sitting on the beach with her dog and a cell phone


In [5]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [57]:
# !ls /content/gdrive/MyDrive/FeatureCreation/mmf_convert_hm_data/data/datasets/hateful_memes/defaults/annotations
# !ls /content/gdrive/MyDrive/FeatureCreation/mmf_convert_hm_data/data/datasets/hateful_memes/defaults/images/img

In [ ]:
!pip install jsonlines
!mkdir /content/gdrive/MyDrive/FeatureCreation/features/captions

In [55]:
import json
IMG_BASE = "/content/gdrive/MyDrive/FeatureCreation/mmf_convert_hm_data/data/datasets/hateful_memes/defaults/images"
ANN_BASE = "/content/gdrive/MyDrive/FeatureCreation/mmf_convert_hm_data/data/datasets/hateful_memes/defaults/annotations"


def combine_ann_with_captions(ann_base, ann_file, img_base):
  with open(f"{ANN_BASE}/{ann_file}", mode = 'r') as f:
      annotation_jsons = []
      for i, line in enumerate(f):
          img_ann = json.loads(line)
          img_uri = f"{IMG_BASE}/{img_ann['img']}"

          #generate caption
          raw_image = Image.open(img_uri).convert('RGB')
          inputs = processor(raw_image, return_tensors="pt").to("cuda")
          out = model.generate(**inputs)
          caption = processor.decode(out[0], skip_special_tokens=True)


          img_ann["generated_caption"] = caption
          annotation_jsons.append(img_ann)
          
          if i % 10 == 0:
            print(i)
      return annotation_jsons


CAPTION_DEST = "/content/gdrive/MyDrive/FeatureCreation/features/captions"

for ann_file in ["train.jsonl", "dev.jsonl", "test.jsonl"]:
  ann_updated = combine_ann_with_captions(ANN_BASE, ann_file, IMG_BASE)
  dest_file = f"{CAPTION_DEST}/{ann_file}"
  with jsonlines.open(dest_file, 'w') as writer:
    writer.write_all(ann_updated)

{'id': 42953, 'img': 'img/42953.png', 'label': 0, 'text': 'its their character not their color that matters', 'generated_caption': 'there are two different pictures of a man with a bald head'}
0
{'id': 23058, 'img': 'img/23058.png', 'label': 0, 'text': "don't be afraid to love again everyone is not like your ex", 'generated_caption': 'there is a man and woman that are kissing each other'}
{'id': 13894, 'img': 'img/13894.png', 'label': 0, 'text': 'putting bows on your pet', 'generated_caption': 'there are two pictures of a cat with a red bow on it'}
{'id': 37408, 'img': 'img/37408.png', 'label': 0, 'text': 'i love everything and everybody! except for squirrels i hate squirrels', 'generated_caption': 'there are two pictures of a dog that are looking at the camera'}
{'id': 82403, 'img': 'img/82403.png', 'label': 0, 'text': 'everybody loves chocolate chip cookies, even hitler', 'generated_caption': 'arafed image of a man in a suit and tie holding a chocolate bar'}


KeyboardInterrupt: ignored

In [47]:
# !cat /content/gdrive/MyDrive/FeatureCreation/features/test_captions/dev.jsonl

{"id": 42953, "img": "img/42953.png", "label": 0, "text": "its their character not their color that matters", "generated_caption": "there are two different pictures of a man with a bald head"}
{"id": 23058, "img": "img/23058.png", "label": 0, "text": "don't be afraid to love again everyone is not like your ex", "generated_caption": "there is a man and woman that are kissing each other"}
{"id": 13894, "img": "img/13894.png", "label": 0, "text": "putting bows on your pet", "generated_caption": "there are two pictures of a cat with a red bow on it"}
